# Zap Demo

This notebook demonstrates Zap, a library for distributed processing of chunked NumPy arrays. We'll start with the `direct` engine, which is the simplest engine. Processing is carried out locally, in-memory.

First, create a Zap array filled with integer 1s.

In [1]:
import zap.direct
a = zap.direct.ones((10, 2), chunks=(2, 2), dtype='i4')

We can look at the contents of the array by calling its `asndarray` method.

In [2]:
a.asndarray()

array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]], dtype=int32)

Notice that the chunking is hidden, since it's an implementation detail.

Now we have a Zap array, we can run a computation on it. Zap exposes the standard NumPy API, accessible via the `zap.base` module. Here we simply sum all columns.

In [3]:
import zap.base as np
b = np.sum(a, axis=0)
b.asndarray()

array([10, 10])

We can save an array in Zarr format. If the array does not match the chunk size supplied -- either because it is different to the original chunk size, or the number of rows in some partitions has changed in the course of applying NumPy operations -- then the rows are repartitioned before being written out.

In [4]:
a.to_zarr("/tmp/a.zarr", chunks=(2, 2))

In [5]:
! find /tmp/a.zarr

/tmp/a.zarr
/tmp/a.zarr/.zarray
/tmp/a.zarr/1.0
/tmp/a.zarr/2.0
/tmp/a.zarr/3.0
/tmp/a.zarr/0.0
/tmp/a.zarr/4.0


It's easy to read Zap arrays from Zarr. In this case there's no need to specify a chunk size since the array automatically inherits Zarr's chunk size.

In [6]:
c = zap.direct.from_zarr("/tmp/a.zarr")
np.sum(c, axis=0).asndarray()

array([10, 10])

### Serverless Processing with Pywren

[Pywren](http://pywren.io/) allows you to run Python code at scale using AWS Lambda. Zap has a Pywren engine called `executor` that makes it easy to perform NumPy calculations at scale.

Before starting, [install Pywren](http://pywren.io/pages/gettingstarted.html) in the same Python virtual environment you created to run this demo.

Zap uses a custom Pywren runtime for Zarr support, which is enabled by editing the `runtime` section in *~/.pywren_config* to be:

```
runtime:
    s3_bucket: tom-pywren-runtimes
    s3_key: pywren.runtime/pywren_runtime-3.6-default.meta.json
```

Now that Pywren is set up, we can create a Zap array of 1s, just like for the `direct` engine above. This time we need to pass in a Python [Executor](https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.Executor) object, so we create one using Pywren as follows:

In [7]:
import pywren
import zap.executor
executor = zap.executor.PywrenExecutor()

Using the `executor` we can create the Zap array...

In [8]:
a = zap.executor.ones(executor, (10, 2), chunks=(2, 2), dtype='i4')

... and save it as a Zarr array in S3 cloud storage. (You will need to change the S3 path to be a bucket that you have write access to.)

In [9]:
import s3fs.mapping
s3 = s3fs.S3FileSystem()
path = 'sc-tom-test-data/ones.zarr'
output_zarr = s3fs.mapping.S3Map(path, s3=s3)
a.to_zarr(output_zarr, a.chunks)

This may take tens of seconds to a minute to run. What's happening behind the scenes is that Pywren serializes the execution graph and input data values, then runs each input as an AWS Lambda invocation. The execution graph in this case is the `ones` function that creates an array chunk of 1s, followed by a write operation that writes the array chunk to Zarr in S3. The data values are five `(2, 2)` chunk sizes that are inputs to the `ones` function.

The important point is that the processing all happened in the cloud, not in the local Python process.

The result is that there is a Zarr array stored on S3 as five `(2, 2)` chunks. We can see the raw files (_0.0_, _1.0_, ... _4.0_) by listing the S3 bucket:

In [10]:
s3.ls(path)

['sc-tom-test-data/ones.zarr/.zarray',
 'sc-tom-test-data/ones.zarr/0.0',
 'sc-tom-test-data/ones.zarr/1.0',
 'sc-tom-test-data/ones.zarr/2.0',
 'sc-tom-test-data/ones.zarr/3.0',
 'sc-tom-test-data/ones.zarr/4.0']

Finally, delete the data from S3:

In [11]:
s3.rm(path, recursive=True)